# Analyze and Visualize NBA Team Ratings 

In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/teams/"
box_DIR = "../data/box/"

# Net Rating Variance

In [ ]:
season = 2024

In [ ]:
df1 = pd.read_parquet(box_DIR + f"NBA_Box_T_Adv_{season}.parquet")
cols = [
    "gameId",
    "teamId",
    "offensiveRating",
    "defensiveRating",
    "netRating",
    "possessions",
]
df1 = df1[cols]
df1["Win"] = df1["netRating"] > 0
df1["Loss"] = df1["netRating"] < 0
stats = leaguegamelog.LeagueGameLog(
    player_or_team_abbreviation="T",
    season=season,
    season_type_all_star="Regular Season",
)
df2 = stats.get_data_frames()[0]

df2 = df2.rename(
    columns={
        "GAME_ID": "gameId",
        "TEAM_ID": "teamId",
        "TEAM_ABBREVIATION": "teamTricode",
        "GAME_DATE": "gameDate",
        "TEAM_NAME": "teamName",
    }
)
cols2 = ["gameId", "teamId", "teamTricode", "gameDate"]
df2 = df2[cols2]
df2["gameId"] = df2["gameId"].astype(int)
df3 = pd.merge(df2, df1, on=["gameId", "teamId"])
df4 = df3.sort_values(by=["teamTricode", "gameDate"]).reset_index(drop=True)
data = df4
data = data.rename(columns={"gameDate": "Date"})

In [ ]:
grp = data.groupby("teamTricode")["netRating"]
sorted_grp = grp.describe().sort_values(by="std", ascending=False).reset_index()
teams_sorted = sorted_grp["teamTricode"].tolist()
sorted_grp = sorted_grp.round(2)
sorted_grp.head()
sorted_grp = sorted_grp.set_index("teamTricode")
sorted_grp = sorted_grp.reset_index()

In [ ]:
teams_cat = pd.Categorical(data["teamTricode"], categories=teams_sorted)
df1 = data.assign(teams_cat=teams_cat)
df = add_tinfo(df1,on="teamTricode")

In [ ]:
var, y = "netRating", "Net Rating"
title = "NBA" + " " + y + " " + "Distribution" + " " + get_ss(season)
p = (
    ggplot(df)  # What data to use
    + aes(x=var, group="teams_cat", fill="colorsTeam", image="image")  # What variable to use
    + stat_density(
        aes(y=after_stat("scaled")),
        show_legend=False,
        alpha=0.5,
    )
    + geom_image(aes(x=-18,y=0.85),size=0.07)
    + geom_vline(xintercept=0,color="black")
    + scale_color_identity(aesthetics=["fill"])
    + coord_cartesian(xlim=[-20,20])
    + scale_x_continuous(breaks=[-20, -10, 0, 10, 20])
    + facet_wrap(facets="~ teams_cat")
    + labs(
        x=y,
        y="Frequency",
        title=title,
        subtitle="x-axis shows the points and y-axis shows fraction of games with that Net Rating | sorted by highest std deviation to least",
    )
    + theme_xkcd(base_size=12,stroke_size=0.5)
       + theme(
        plot_title=element_text(face="bold", size=24),
        plot_subtitle=element_text(size=14),
        plot_caption=element_text(size=12),
        plot_margin=0.025,
        axis_title_x=element_text(face="bold", size=16),
        axis_title_y=element_text(face="bold", size=16),
        axis_text_y=element_blank(),
        figure_size=(15,10),
        dpi=200,
        text=element_text(family="Comic Sans MS"),
        strip_align=0,
    )
    + pnba
)
# p.save(fig_DIR + "NBA" + "_" + var + ".png", verbose=False, dpi=300, width=15, height=10)
p.draw()

In [ ]:
var, y = "netRating", "Net Rating"
title = "NBA" + " " + y + " " + "Distribution" + " " + get_ss(season)
p = (
    ggplot(df)  # What data to use
    + aes(x=var, color="colorsTeam", fill="colorsTeam")  # What variable to use
    + stat_bin(
        aes(y=after_stat("ndensity")),
        show_legend=False,
        binwidth=5,
        alpha=0.2,
        color="black",
    )
    + stat_density(
        aes(y=after_stat("scaled")),
        show_legend=False,
        alpha=0,
        size=3
    )
    + geom_vline(xintercept=0,color="black")
    + scale_color_identity(aesthetics=["fill","color"])
    + coord_cartesian(xlim=[-20,20])
    + scale_x_continuous(breaks=[-20, -10, 0, 10, 20])
    + facet_wrap(facets="~ teams_cat")
    + labs(
        x=y,
        y="Frequency",
        title=title,
        subtitle="x-axis shows the points and y-axis shows fraction of games with that Net Rating | sorted by highest std deviation to least",
    )
    + theme_xkcd(base_size=12,stroke_size=0.5)
    + theme(
        plot_title=element_text(face="bold", size=24),
        plot_subtitle=element_text(size=14),
        plot_caption=element_text(size=12),
        plot_margin=0.025,
        axis_title_x=element_text(face="bold", size=16),
        axis_title_y=element_text(face="bold", size=16),
        axis_text_y=element_blank(),
        figure_size=(15,10),
        dpi=200,
        text=element_text(family="Comic Sans MS"),
        strip_align=0,
    )
    + pnba
)
p

In [ ]:

# p.save(fig_DIR + "NBA" + "_" + var + ".png", verbose=False, dpi=300, width=15, height=10)
p.draw()

In [ ]:
var, y = "netRating", "Net Rating"
# var,y = "Minutes", "Minutes Played"
title = "NBA" + " " + y + " " + "Boxplot"
p = (
    ggplot(df)  # What data to use
    + aes(x="teams_cat", y=var, fill="colorsTeam")  # What variable to use
    # + geom_line()  # Geometric object to use for drawing
    + geom_boxplot(alpha=0.5)
    + scale_color_identity(aesthetics=["fill"])
    + scale_y_continuous(limits=[-20, 20], breaks=[-20, -10, 0, 10, 20])
    # + geom_hline(yintercept=0.752, linetype="dashed", color="blue", size=0.5)
    # + annotate("text", x=df["gameDate"].iloc[1], y=0.76, label="League Avg")
    + labs(x="Team", y=y, title=title, caption="@sradjoker | source: nba.com/stats")
    # + themes.theme_xkcd(base_size=14)
    + themes.theme_538(base_size=12)
    + theme(plot_title=element_text(face="bold", size=18), plot_margin=0.025)
)
# p.save(fig_DIR + "NBA" + "_" + var + "_boxplot.png", dpi=300, height=5, width=15)
p.draw()

# Net Rating Variance Multiple Seasons

In [ ]:
team = "Boston Celtics"
df1 = get_box("T","Adv",False,[2023,2024])
df1["team_name"]=df1["teamcity"] + " " + df1["teamname"]
df2 = get_box("T","Adv",True,[2023,2024])
df1 = df1.query(f"team_name == '{team}'")
df2 = df2.query(f"team_name == '{team}'")
data = add_tinfo(df1)

In [ ]:
var, y = "netrating", "Net Rating"
title = team + " " + y + " " + "Distribution 2024 vs 2025"
p = (
    ggplot(data)  # What data to use
    + aes(x=var, color="colorsTeam", fill="colorsTeam")  # What variable to use
    + stat_bin(
        aes(y=after_stat("ndensity")),
        show_legend=False,
        binwidth=5,
        alpha=0.2,
        color="black",
    )
    + stat_density(
        aes(y=after_stat("scaled")),
        show_legend=False,
        alpha=0,
        size=3
    )
    + geom_vline(xintercept=0,color="black",linetype="dashed",size=1.5)
    + scale_color_identity(aesthetics=["fill","color"])
    + coord_cartesian(xlim=[-20,20])
    + scale_x_continuous(breaks=[-20, -10, 0, 10, 20])
    + facet_wrap(facets="~ season")
    + labs(
        x=y,
        y="Frequency",
        title=title,
        subtitle=f"Net Rtg 2024:{df2["net_rating"].iloc[0]}, 2025:{df2["net_rating"].iloc[1]}"
    )
    + theme_xkcd(base_size=10,stroke_size=0.5)
    + theme(
        plot_title=element_text(face="bold",size=16),
        plot_margin=0.025,
        axis_title_x=element_text(face="bold"),
        axis_title_y=element_text(face="bold"),
        axis_text_y=element_blank(),
        figure_size=(7,5),
        dpi=200,
        text=element_text(family="Comic Sans MS"),
        strip_align=0,
    )
    + pnba
)
p